In [56]:
import pandas as pd
import nltk
import spacy
from nltk import word_tokenize
from nltk.corpus import wordnet
import time
from collections import Counter
import numpy as np
collision_data_preventable= pd.read_csv("../data/Clean_data/Collision_preventable.csv")
claim_data =  pd.read_csv("../data/TransLink Raw Data/claim_vehicle_employee_line.csv")


/home/simardeep/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
pd.read_csv("../data/TransLink Raw Data/Preventable and Non Preventable_tabDelimited.txt", delimiter='\t')

,occurrence_id,claim_id,claim_desc,Loss_Date,Time_of_Loss,Loss_Location_At,Loss_Location_On,City_of_Incident,Preventable_NonPreventable,Cost_Centre,...,Vehicle_Number,CMBC_Seniority_Number,Description_Code_1,Description_Code_1_Desc,APTA,APTA_Desc,Asset_Vehicle_Year,Asset_Manufacturer_Model,Asset_Manufacturer,Status
0,9808,171117,BUS SLID INTO LIGHT POLE IN SNOW,01/01/2011,2230,WINDSOR,41 AVE,VAN,N,167,...,9494,72325,A180,COLLISION WITH FIXED OBJECT,CO,Collisions with Objects,NaN,NaN,NaN,FINAL
1,9664,170719,BUS 3337 HIT REAR OF COACH 9534,03/01/2011,1020,22nd St Station Loop,Ladner St,New Westminster,P,171,...,3337,43348,A160,COLL IN GARAGE/ LOOPS & FIXED OBJ,CV,Collisions with Other Vehicles,NaN,NaN,NaN,OPEN
2,9667,170722,COACH HIT POLE WITH R/SIDE MIRROR,03/01/2011,1405,NANTON,GRANVILLE,VAN,P,167,...,2124,82761,A182,"FIXED OBJECT, RIGHT MIRROR ONLY",CO,Collisions with Objects,NaN,NaN,NaN,OPEN
3,9672,170727,L/SIDE OF 9480 SCRAPED R/REAR BUMPER OF 9635,03/01/2011,1815,NaN,BTC YARD,BUR,P,169,...,9480,77278,A160,COLL IN GARAGE/ LOOPS & FIXED OBJ,CV,Collisions with Other Vehicles,NaN,NaN,NaN,OPEN
4,9713,169943,BUS S363 SLID INTO R/BUMPER OF BUS S366,03/01/2011,1020,BIDWELL,DAVIE,VAN,N,366,...,S363,82417,A100,OV AHEAD OF RV,CV,Collisions with Other Vehicles,NaN,NaN,NaN,OPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23800,201738,257687,TP CLIPPED STATIONARY BUS LF MIRROR- NO DMG,24/04/2020,0935,W 33 Ave,Granville St,Vancouver,N,167,...,2208,85533,A125,OV CUTS IN SCRAPES L SIDE RV,CV,Collisions with Other Vehicles,2006.0,E40LF,WF,OPEN
23801,201725,257668,BUS BIKE RACK REAR ENDED TP,24/04/2020,1655,74 Ave,Scott Rd,Surrey,P,170,...,18115,83172,A141,VEH APPROACH FROM ANGLE - LEFT,CV,Collisions with Other Vehicles,2018.0,XN40,WF,OPEN
23802,201741,257691,H&R TP STRUCK BUS RR CORNER,25/04/2020,1613,105 Ave,152 St,Surrey,N,170,...,18032,58465,A142,VEH APPROACH FROM ANGLE - RIGHT,CV,Collisions with Other Vehicles,2018.0,XDE60,WF,OPEN
23803,201728,257672,BUS LR CORNER BACKED INTO FENCE,25/04/2020,1838,NaN,Surrey Central Bus Loop,Surrey,P,170,...,18152,84733,A181,VEH HIT OBJCT BACKING/FALL BACK,CO,Collisions with Objects,2018.0,XN40,WF,OPEN


In [2]:
Collision_preventable = pd.read_excel('../data/TransLink Raw Data/2020 Collisions- Preventable and Non Preventable UBC Set Without Claim Number.xlsx', skiprows=  3)

In [3]:
location_data = pd.read_csv("../data/TransLink Raw Data/collision_locations_with_coordinates.csv")

In [4]:
count_df =  collision_data_preventable.groupby(["APTA", "Claim Desc."]).size().reset_index().rename(columns={0:"Count"})

### Checking the count of different events based on broader categories of incidents

In [5]:
count_df.sort_values(by=['Count'], ascending=False)

,APTA,Claim Desc.,Count
13,CO,BUS RF MIRROR HIT POLE,9
26,CO,BUS RR SIDE HIT POST,3
14,CO,BUS RF MIRROR HIT TREE,3
25,CO,BUS RR SIDE HIT POLE,2
85,CV,BUS RR SIDE HIT PARKED TP LF SIDE,2
...,...,...,...
57,CV,BUS 2131 HIT PARKED TP VEH,1
58,CV,BUS ALLEGEDLY CONTACTED TP RR CORNER - NO DMG,1
59,CV,BUS BIKE RACK REAR ENDED TP - NO DMG,1
60,CV,"BUS CHANGED LANES, RS REAR WHEEL HIT & CYCLIST...",1


In [6]:
# loading model of spacy
nlp= spacy.load("en_core_web_sm")

In [7]:
# loading all the stopwords
all_stopwords = nlp.Defaults.stop_words

In [8]:

list_of_list = list()
for sentence in Collision_preventable['Claim Desc.']:
    
    # declaring list of differrent pos tags
    intermediate_list = list()
    pos_list = list()
    noun_list = list()
    description_list = list()
    preposition_list = list()
    verb_list = list()
    adjective_list = list()
    # Removing the digits from the claim descriptions
    sentence = sentence.strip()
    sentence = ''.join([i for i in sentence if not i.isdigit()])
    sentence = sentence.replace('- NO DMG',' ')
    sentence = sentence.replace('-',' ')
    
    text = nltk.word_tokenize(sentence)
    
    
    #Removing stopwords
    
    text_without_sw = [word for word in text if (not word  in all_stopwords or  not word.isdigit())]
    result = ' '.join(text_without_sw)
    spacy_ready_text = nlp(result)
    
    # POS tagging
    for token in spacy_ready_text:
        pos_list.append(token.pos_)
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            noun_list.append(str(token))
        elif token.pos_ == 'VERB':
            verb_list.append(str(token))
        elif token.pos_ == 'PROPN':
            preposition_list.append(str(token))

    tag = 'unknown'        
    chosen_verb_list = list()
    if verb_list == [] and preposition_list != []:
        for w in preposition_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb':
                    chosen_verb_list.append(w)
                else:
                    continue
        if chosen_verb_list == []:
            chosen_verb_list = preposition_list
                    
    elif preposition_list == [] and verb_list != []:
        for w in verb_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb':
                    chosen_verb_list.append(w)
                else:
                    continue
        if chosen_verb_list == []:
            chosen_verb_list = verb_list            
                
                    
    elif verb_list == [] and preposition_list == []:
        for w in noun_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb' or syns[0].lexname().split('.')[0] == 'adj':
                    chosen_verb_list.append(w)
                elif w == 'HIT':
                    chosen_verb_list.append(w)
                else:
                    continue
        
    else:
        for w in verb_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb':
                    chosen_verb_list.append(w)
                else:
                    continue
                    
    if chosen_verb_list == [] and verb_list != []:
        chosen_verb_list = verb_list

    elif chosen_verb_list == [] and preposition_list != []:
        chosen_verb_list = preposition_list
        
    
    impact_list = []
    if noun_list:
        impact_list.append(noun_list[-1])


    
        

        
    intermediate_list.extend([sentence, pos_list,preposition_list,verb_list, chosen_verb_list,pos_list, noun_list, impact_list ])
    list_of_list.append(intermediate_list)    
pos_df = pd.DataFrame(list_of_list, columns=['Description','POS','preposition','verb', 'chosen_verb', 'pos','noun','impact'])


    

In [9]:
pos_df['impact'] = pos_df['impact'].apply(','.join).str.lower()

In [10]:
pos_df['chosen_verb'] = pos_df['chosen_verb'].apply(','.join).str.lower()

In [11]:
pos_df['impact'] = pos_df['impact'].replace(['rr','rf','rl','lf','rd','rs','s','l','ls'], 'Side of the vehicle', regex= False)

In [12]:
pos_df['impact'] = pos_df['impact'].replace(['dmgd','dmged',], 'Damaged', regex= False)

In [13]:
pos_df['impact'] = pos_df['impact'].replace(['veh'], 'Vehicle', regex= False)

In [14]:
pos_df['impact'] = pos_df['impact'].replace(['tp'], 'Third party', regex= False)

In [15]:
lst = ["BUS", "BUS", "TROLLEY", "POLES", "CUT", "TROLLEY", "ROPE"]

In [16]:
checker_verb = dict()
for i in pos_df['chosen_verb']:
    if ',' in i:
        action_list = i.split(',')
        for word in action_list:
            
            if word.lower() not in checker_verb:
                checker_verb[word.lower()]= 1
            else:
                checker_verb[word.lower()]+= 1
    else:
        if i.lower() not in checker_verb:
            checker_verb[i.lower()]= 1
        else:
            checker_verb[i.lower()]+= 1         

In [17]:
checker = dict()
for i in pos_df['impact']:
        if i.lower() not in checker:
            checker[i.lower()]= 1
        else:
            checker[i.lower()]+= 1         

In [18]:
Impacted_object_df = pd.DataFrame(checker.items(), columns= ['Impacted Object','Count']).sort_values(by='Count', ascending=False)
Impacted_object_df.to_excel("../data/TransLink Raw Data/Impacted_object_count.xlsx", index=False)
Verbs_df = pd.DataFrame(checker.items(), columns= ['Verbs','Count']).sort_values(by='Count', ascending=False)
Verbs_df.to_excel("../data/TransLink Raw Data/Verb_count.xlsx", index= False)

In [18]:
result_df = pd.concat([collision_data_preventable['Loss Date (MM/DD/YEAR)'], collision_data_preventable['Claim Desc.'], pos_df['chosen_verb'],pos_df['pos'],pos_df['noun'], pos_df['impact'],location_data['latt'], location_data['long']],axis = 1)

In [19]:
result_df = result_df.rename(columns={'Loss Date (MM/DD/YEAR)': 'Date of Incident'})

In [20]:
result_len = len(result_df)

In [21]:
result_df['claim_id']= np.arange(1, result_len+1, 1)

In [77]:
#list_of_colour = pd.read_csv("../data/TransLink Raw Data/color/ral_standard.csv")[[' English']]
list_of_colour = list_of_colour.rename(columns={' English': 'name'})

In [23]:
list_of_colour = pd.read_json("../data/TransLink Raw Data/data.json")[['name']]

In [24]:
result_df

,Date of Incident,Claim Desc.,chosen_verb,pos,noun,impact,latt,long,claim_id
0,2020-03-26,TURNING BUS R/S HIT POLE,turning,"[VERB, PROPN, PROPN, SYM, NOUN, NOUN, NOUN]","[BUS, R, S, HIT, POLE]",pole,49.279850,-123.110585,1
1,2020-03-25,"BUS UNDERCARRIAGE HIT CURB, TOW REQUIRED",hit,"[PROPN, NOUN, VERB, NOUN, PUNCT, PROPN, PROPN]","[BUS, UNDERCARRIAGE, CURB, TOW, REQUIRED]",required,49.309633,-123.083554,2
2,2020-03-24,BUS RAN RED LIGHT AND LEFT FRONT CORNER HIT TP...,hit,"[PROPN, PROPN, PROPN, PROPN, CCONJ, PROPN, NOU...","[BUS, RAN, RED, LIGHT, LEFT, FRONT, CORNER, TP...",onboard,49.233129,-123.105658,3
3,2020-03-23,BUS RR SIDE STRUCK PARKED TP RR SIDE,struck,"[NOUN, NOUN, PROPN, VERB, PROPN, NOUN, NOUN, P...","[BUS, RR, SIDE, PARKED, TP, RR, SIDE]",side,49.245457,-123.139213,4
4,2020-03-21,SHUTTLE RF MIRROR CONTACTED PARKED TP LF MIRROR,contacted,"[PROPN, PROPN, PROPN, VERB, PROPN, NOUN, NOUN,...","[SHUTTLE, RF, MIRROR, PARKED, TP, LF, MIRROR]",mirror,49.286629,-122.814832,5
...,...,...,...,...,...,...,...,...,...
171,2020-01-05,BUS RF MIRROR HIT POLE,hit,"[PROPN, PROPN, PROPN, VERB, NOUN]","[BUS, RF, MIRROR, POLE]",pole,49.245457,-123.139213,172
172,2020-01-05,BUS RF CORNER HIT PARKED TP LF MIRROR - NO DMG,hit,"[PROPN, PROPN, PROPN, VERB, PROPN, NOUN, NOUN,...","[BUS, RF, CORNER, PARKED, TP, LF, MIRROR]",mirror,49.232712,-122.909670,173
173,2020-01-04,BUS/BUS 2198 TROLLEY POLE CLIPPED STATIONARY 2...,"clipped,stationary","[PROPN, SYM, NOUN, NOUN, NOUN, NOUN, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, CLIPPED, STATIONARY,...",rope,49.286007,-123.126981,174
174,2020-01-03,BUS/BUS 2208 TROLLEY POLE HIT STATIONARY 2543 ...,hit,"[PROPN, SYM, NOUN, NOUN, NOUN, VERB, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, STATIONARY, TROLLEY,...",rope,49.224264,-123.164132,175


In [25]:
intermediate_df = (result_df[['claim_id','chosen_verb']].set_index(['claim_id'])
  .apply(lambda x: x.str.split(',').explode()).reset_index()
 
)
result_verb_df = pd.merge(intermediate_df, result_df, on= 'claim_id')



In [26]:
impact_list = list(np.unique(result_df['impact']))
impact_colour=list()
for i in result_df['impact']:
    impact_colour.append(list_of_colour.iloc[impact_list.index(i)]['name'])
    

In [27]:
verb_list = list(np.unique(result_verb_df['chosen_verb_x']))
verb_colour=list()
for i in result_verb_df['chosen_verb_x']:
    verb_colour.append(list_of_colour.iloc[verb_list.index(i)]['name'])

In [28]:
result_df['impact_colour']= impact_colour
result_verb_df['verb_colour']= verb_colour

In [38]:
impact_colour

['DeepSkyBlue2',
 'SpringGreen2',
 'DeepSkyBlue3',
 'DarkTurquoise',
 'Turquoise4',
 'DeepSkyBlue2',
 'DeepPink4',
 'DarkTurquoise',
 'DarkTurquoise',
 'Cyan3',
 'DarkTurquoise',
 'Turquoise4',
 'DarkTurquoise',
 'Aqua',
 'Grey',
 'Aqua',
 'DarkTurquoise',
 'Green3',
 'DarkTurquoise',
 'DeepSkyBlue2',
 'Red',
 'Turquoise4',
 'DarkBlue',
 'Aqua',
 'Yellow',
 'Green',
 'DarkTurquoise',
 'DeepSkyBlue3',
 'Turquoise4',
 'Blue1',
 'DarkGreen',
 'Cyan2',
 'DarkTurquoise',
 'Maroon',
 'Maroon',
 'DarkTurquoise',
 'Turquoise4',
 'DeepSkyBlue2',
 'Turquoise4',
 'DarkTurquoise',
 'SpringGreen2',
 'Blue',
 'DarkTurquoise',
 'Turquoise4',
 'Lime',
 'Turquoise4',
 'Aqua',
 'DeepSkyBlue2',
 'Turquoise4',
 'Cyan3',
 'DarkTurquoise',
 'DeepSkyBlue4',
 'SpringGreen1',
 'Aqua',
 'Green1',
 'Aqua',
 'LightSeaGreen',
 'Turquoise4',
 'SpringGreen1',
 'Green3',
 'Purple',
 'SpringGreen1',
 'DeepSkyBlue4',
 'DeepSkyBlue2',
 'SpringGreen3',
 'DodgerBlue1',
 'Turquoise4',
 'Turquoise2',
 'Blue3',
 'MediumSprin

In [49]:
result_df = result_df.replace('Silver','skyblue')
result_df = result_df.replace('Aqua','Pink')
result_df = result_df.replace('Fuchsia','Red')
result_df = result_df.replace('Lime','peachpuff')
result_df = result_df.replace('Olive','orange')
result_df = result_df.replace('Teal','yellowgreen')
result_df = result_df.replace('SpringGreen3','SpringGreen')
result_df = result_df.replace('Cyan3','Cyan')
result_df = result_df.replace('DodgerBlue3','DodgerBlue')
result_df = result_df.replace('DodgerBlue1','darkmagenta')
result_df = result_df.replace('DodgerBlue2','gold')
result_df = result_df.replace('DeepSkyBlue1','firebrick')
result_df = result_df.replace('DeepSkyBlue2','DeepSkyBlue')
result_df = result_df.replace('DeepSkyBlue3','thistle')
result_df = result_df.replace('DeepSkyBlue4','tan')
result_df = result_df.replace('Green1','forestgreen')
result_df = result_df.replace('Green3','Green')
result_df = result_df.replace('Green4','plum')
result_df = result_df.replace('Turquoise3','rosybrown')
result_df = result_df.replace('Turquoise4','royalblue')
result_df = result_df.replace('Turquoise2','darkorange')
result_df = result_df.replace('SpringGreen2','darksalmon')
result_df = result_df.replace('SpringGreen1','darkgoldenrod')
result_df = result_df.replace('DeepPink4','darkkhaki')
result_df = result_df.replace('Blue3','chocolate')
result_df = result_df.replace('Blue1','darkorchid')


In [52]:
result_verb_df = result_verb_df.replace('Silver','skyblue')
result_verb_df = result_verb_df.replace('Aqua','Pink')
result_verb_df = result_verb_df.replace('Fuchsia','Red')
result_verb_df = result_verb_df.replace('Lime','peachpuff')
result_verb_df = result_verb_df.replace('Olive','orange')
result_verb_df = result_verb_df.replace('Teal','yellowgreen')
result_verb_df = result_verb_df.replace('SpringGreen3','SpringGreen')
result_verb_df = result_verb_df.replace('Cyan3','Cyan')
result_verb_df = result_verb_df.replace('DodgerBlue3','DodgerBlue')
result_verb_df = result_verb_df.replace('DodgerBlue1','darkmagenta')
result_verb_df = result_verb_df.replace('DodgerBlue2','gold')
result_verb_df = result_verb_df.replace('DeepSkyBlue1','firebrick')
result_verb_df = result_verb_df.replace('DeepSkyBlue2','DeepSkyBlue')
result_verb_df = result_verb_df.replace('DeepSkyBlue3','thistle')
result_verb_df = result_verb_df.replace('DeepSkyBlue4','tan')
result_verb_df = result_verb_df.replace('Green1','forestgreen')
result_verb_df = result_verb_df.replace('Green3','Green')
result_verb_df = result_verb_df.replace('Green4','plum')
result_verb_df = result_verb_df.replace('Turquoise3','rosybrown')
result_verb_df = result_verb_df.replace('Turquoise4','royalblue')
result_verb_df = result_verb_df.replace('Turquoise2','darkorange')
result_verb_df = result_verb_df.replace('SpringGreen2','darksalmon')
result_verb_df = result_verb_df.replace('SpringGreen1','darkgoldenrod')
result_verb_df = result_verb_df.replace('DeepPink4','darkkhaki')
result_verb_df = result_verb_df.replace('Blue3','chocolate')
result_verb_df = result_verb_df.replace('Blue1','darkorchid')


In [53]:
result_df

,Date of Incident,Claim Desc.,chosen_verb,pos,noun,impact,latt,long,claim_id,impact_colour
0,2020-03-26,TURNING BUS R/S HIT POLE,turning,"[VERB, PROPN, PROPN, SYM, NOUN, NOUN, NOUN]","[BUS, R, S, HIT, POLE]",pole,49.279850,-123.110585,1,DeepSkyBlue
1,2020-03-25,"BUS UNDERCARRIAGE HIT CURB, TOW REQUIRED",hit,"[PROPN, NOUN, VERB, NOUN, PUNCT, PROPN, PROPN]","[BUS, UNDERCARRIAGE, CURB, TOW, REQUIRED]",required,49.309633,-123.083554,2,darksalmon
2,2020-03-24,BUS RAN RED LIGHT AND LEFT FRONT CORNER HIT TP...,hit,"[PROPN, PROPN, PROPN, PROPN, CCONJ, PROPN, NOU...","[BUS, RAN, RED, LIGHT, LEFT, FRONT, CORNER, TP...",onboard,49.233129,-123.105658,3,thistle
3,2020-03-23,BUS RR SIDE STRUCK PARKED TP RR SIDE,struck,"[NOUN, NOUN, PROPN, VERB, PROPN, NOUN, NOUN, P...","[BUS, RR, SIDE, PARKED, TP, RR, SIDE]",side,49.245457,-123.139213,4,DarkTurquoise
4,2020-03-21,SHUTTLE RF MIRROR CONTACTED PARKED TP LF MIRROR,contacted,"[PROPN, PROPN, PROPN, VERB, PROPN, NOUN, NOUN,...","[SHUTTLE, RF, MIRROR, PARKED, TP, LF, MIRROR]",mirror,49.286629,-122.814832,5,royalblue
...,...,...,...,...,...,...,...,...,...,...
171,2020-01-05,BUS RF MIRROR HIT POLE,hit,"[PROPN, PROPN, PROPN, VERB, NOUN]","[BUS, RF, MIRROR, POLE]",pole,49.245457,-123.139213,172,DeepSkyBlue
172,2020-01-05,BUS RF CORNER HIT PARKED TP LF MIRROR - NO DMG,hit,"[PROPN, PROPN, PROPN, VERB, PROPN, NOUN, NOUN,...","[BUS, RF, CORNER, PARKED, TP, LF, MIRROR]",mirror,49.232712,-122.909670,173,royalblue
173,2020-01-04,BUS/BUS 2198 TROLLEY POLE CLIPPED STATIONARY 2...,"clipped,stationary","[PROPN, SYM, NOUN, NOUN, NOUN, NOUN, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, CLIPPED, STATIONARY,...",rope,49.286007,-123.126981,174,Cyan
174,2020-01-03,BUS/BUS 2208 TROLLEY POLE HIT STATIONARY 2543 ...,hit,"[PROPN, SYM, NOUN, NOUN, NOUN, VERB, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, STATIONARY, TROLLEY,...",rope,49.224264,-123.164132,175,Cyan


In [54]:

result_df.to_excel("../data/TransLink Raw Data/Claim_colour_df.xlsx", index=False)

In [55]:

result_verb_df.to_excel("../data/TransLink Raw Data/verb_colour_df.xlsx", index=False)

In [50]:
result_verb_df

,claim_id,chosen_verb_x,Date of Incident,Claim Desc.,chosen_verb_y,pos,noun,impact,latt,long,impact_colour,verb_colour
0,1,turning,2020-03-26,TURNING BUS R/S HIT POLE,turning,"[VERB, PROPN, PROPN, SYM, NOUN, NOUN, NOUN]","[BUS, R, S, HIT, POLE]",pole,49.279850,-123.110585,DeepSkyBlue2,SpringGreen2
1,2,hit,2020-03-25,"BUS UNDERCARRIAGE HIT CURB, TOW REQUIRED",hit,"[PROPN, NOUN, VERB, NOUN, PUNCT, PROPN, PROPN]","[BUS, UNDERCARRIAGE, CURB, TOW, REQUIRED]",required,49.309633,-123.083554,SpringGreen2,Blue1
2,3,hit,2020-03-24,BUS RAN RED LIGHT AND LEFT FRONT CORNER HIT TP...,hit,"[PROPN, PROPN, PROPN, PROPN, CCONJ, PROPN, NOU...","[BUS, RAN, RED, LIGHT, LEFT, FRONT, CORNER, TP...",onboard,49.233129,-123.105658,DeepSkyBlue3,Blue1
3,4,struck,2020-03-23,BUS RR SIDE STRUCK PARKED TP RR SIDE,struck,"[NOUN, NOUN, PROPN, VERB, PROPN, NOUN, NOUN, P...","[BUS, RR, SIDE, PARKED, TP, RR, SIDE]",side,49.245457,-123.139213,DarkTurquoise,DeepSkyBlue1
4,5,contacted,2020-03-21,SHUTTLE RF MIRROR CONTACTED PARKED TP LF MIRROR,contacted,"[PROPN, PROPN, PROPN, VERB, PROPN, NOUN, NOUN,...","[SHUTTLE, RF, MIRROR, PARKED, TP, LF, MIRROR]",mirror,49.286629,-122.814832,Turquoise4,Yellow
...,...,...,...,...,...,...,...,...,...,...,...,...
215,173,hit,2020-01-05,BUS RF CORNER HIT PARKED TP LF MIRROR - NO DMG,hit,"[PROPN, PROPN, PROPN, VERB, PROPN, NOUN, NOUN,...","[BUS, RF, CORNER, PARKED, TP, LF, MIRROR]",mirror,49.232712,-122.909670,Turquoise4,Blue1
216,174,clipped,2020-01-04,BUS/BUS 2198 TROLLEY POLE CLIPPED STATIONARY 2...,"clipped,stationary","[PROPN, SYM, NOUN, NOUN, NOUN, NOUN, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, CLIPPED, STATIONARY,...",rope,49.286007,-123.126981,Cyan3,dark green
217,174,stationary,2020-01-04,BUS/BUS 2198 TROLLEY POLE CLIPPED STATIONARY 2...,"clipped,stationary","[PROPN, SYM, NOUN, NOUN, NOUN, NOUN, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, CLIPPED, STATIONARY,...",rope,49.286007,-123.126981,Cyan3,DeepSkyBlue2
218,175,hit,2020-01-03,BUS/BUS 2208 TROLLEY POLE HIT STATIONARY 2543 ...,hit,"[PROPN, SYM, NOUN, NOUN, NOUN, VERB, NOUN, NOU...","[BUS, BUS, TROLLEY, POLE, STATIONARY, TROLLEY,...",rope,49.224264,-123.164132,Cyan3,Blue1
